In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
!pip install -q -U keras-tuner

In [3]:
import keras_tuner as kt

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [6]:
# Hypermodel
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [8]:
# Hyperband is a hyperparameter tuning algorithm that aims to find the best configuration for your model efficiently.
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

Running multiple configurations in parallel: It starts by randomly sampling different hyperparameter values.
Early stopping: It stops training models that perform poorly early on, saving time and resources.
Adaptive allocation: It allocates more resources to promising configurations and fewer resources to less promising ones.
Iterative process: It repeats these steps multiple times to refine its search.


In [9]:
# Create a callback to stop training early after reaching a certain value for the validation loss.
# if the validation loss does not improve for 5 consecutive epochs, the training will be stopped early.
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

stop_early:
Prevents overfitting: By stopping training early, you can prevent the model from memorizing the training data too much and improve its ability to generalize to unseen data.
Saves time and resources: Training large models can be time-consuming and resource-intensive. EarlyStopping helps you find a good model without wasting time training for unnecessary epochs.

In [11]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.4924 - accuracy: 0.8245 - val_loss: 0.4265 - val_accuracy: 0.8461
Epoch 2/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3726 - accuracy: 0.8644 - val_loss: 0.3788 - val_accuracy: 0.8612
Epoch 3/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3338 - accuracy: 0.8772 - val_loss: 0.3435 - val_accuracy: 0.8777
Epoch 4/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3085 - accuracy: 0.8855 - val_loss: 0.3620 - val_accuracy: 0.8703
Epoch 5/50
1500/1500 [==============================] - 8s 6ms/step - loss: 0.2869 - accuracy: 0.8932 - val_loss: 0.3262 - val_accuracy: 0.8802
Epoch 6/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2701 - accuracy: 0.8990 - val_loss: 0.3197 - val_accuracy: 0.8852
Epoch 7/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2590 - accuracy: 0.9029 - val_loss: 0.3118 - val_accuracy:

In [13]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.4912 - accuracy: 0.8256 - val_loss: 0.3845 - val_accuracy: 0.8622
Epoch 2/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3684 - accuracy: 0.8660 - val_loss: 0.3774 - val_accuracy: 0.8627
Epoch 3/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3300 - accuracy: 0.8796 - val_loss: 0.3674 - val_accuracy: 0.8657
Epoch 4/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3062 - accuracy: 0.8865 - val_loss: 0.4176 - val_accuracy: 0.8503
Epoch 5/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2840 - accuracy: 0.8952 - val_loss: 0.3420 - val_accuracy: 0.8779
Epoch 6/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2696 - accuracy: 0.9005 - val_loss: 0.3377 - val_accuracy: 0.8800
Epoch 7/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2553 - accuracy: 0.9051 - val_loss: 0.3143 - val_accuracy:

In [14]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 2ms/step - loss: 0.5919 - accuracy: 0.8911
[test loss, test accuracy]: [0.591909646987915, 0.8910999894142151]
